In [1]:
import os
import sys
import datetime
import math
import pandas as pd
import numpy as np
import quant_utils.data_moudle as dm
from quant_utils.constant import DB_CONN_JJTG_DATA, DB_CONN_JY, DB_CONN_JY_LOCAL
from fund_db.fund_evaluation_report import generate_fund_report
from quant_utils.df2wordtable import DF2WordTable

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from joblib import Parallel, delayed

In [2]:
query_sql = """
    WITH a AS (
	SELECT
		a.TICKER_SYMBOL,
		a.TRANCHE,
		b.CATEGORY_NAME,
	    CASE
			WHEN b.CATEGORY_NAME != '货币型' AND c.IF_IN_TRANCHE = 1 
            THEN b.ticker_symbol_m 
            ELSE a.TICKER_SYMBOL END AS '代码' 
		FROM
			portfolio_basic_products a
			JOIN fund_info b ON a.TICKER_SYMBOL = b.TICKER_SYMBOL
			LEFT JOIN portfolio_basic_products c ON b.ticker_symbol_m = c.TICKER_SYMBOL 
		WHERE
			1 = 1 
			AND a.IF_IN_TRANCHE = 1 
			AND b.EXPIRE_DATE IS NULL 
		),
		b AS ( SELECT DISTINCT `代码`, tranche AS `备选池` FROM a ) SELECT
		b.*,
		0 AS '是否新入池',
	    CASE
			WHEN d.LEVEL_SUM IN ( '指数', "商品" ) THEN
			'指数和商品' 
			WHEN c.LEVEL_2 = '指数增强' THEN
			'指数和商品' ELSE d.LEVEL_SUM 
		END AS '类别' 
	FROM
		b
		JOIN fund_type_own c ON b.`代码` = c.ticker_symbol
		JOIN fund_type_sum d ON d.LEVEL_1 = c.LEVEL_1 
		AND d.LEVEL_2 = c.LEVEL_2 
	WHERE
		1 = 1 
		AND c.report_date = (
		SELECT
			max( report_date ) 
		FROM
			fund_type_own 
		WHERE
		    report_date < '20240425' 
	)
"""
df = DB_CONN_JJTG_DATA.exec_query(query_sql)

In [2]:
query_sql = """
SELECT
	a.TICKER_SYMBOL AS '代码',
	'优选池' AS '备选池',
	'指数' AS '类别',
	0 AS '是否新入池',
	'20240619' AS '报告日期' 
FROM
	fund_type_own a
	JOIN fund_info b ON a.TICKER_SYMBOL = b.TICKER_SYMBOL 
WHERE
	`LEVEL_1` = '权益指数' 
	AND `REPORT_DATE` = '20240331' 
	AND b.IS_MAIN = 1
"""
df = DB_CONN_JJTG_DATA.exec_query(query_sql)


In [3]:
error_list = []
result_list = []
counter = 0

a = Parallel(n_jobs=-1)(
    delayed(generate_fund_report)(
        ticker_symbol=val['代码'].values[0], 
        tranche=val['备选池'].values[0], 
        if_new=val['是否新入池'].values[0], 
        end_date= val["报告日期"].values[0],
        include_out_tranche=1
    )
    for _, val in df.groupby(by=['代码'])
)
a = pd.concat(a)
a.to_excel("e:/持续评估/基金结果20240619.xlsx")